In [1]:
import connectorx as cx
import pandas as pd
import numpy as np
import os
import duckdb
from datetime import datetime
import glob
import multiprocessing as mp
from functools import partial
import time

In [2]:
def calculate_daily_factor(trading_date):
    date_str = pd.to_datetime(trading_date).strftime('%Y%m%d')
    order_pth = f"/data/cephfs/order/{date_str}.parquet"

    
    conn = duckdb.connect(database=':memory:')

    # 定义六个时间段的查询
    time_intervals = {
        'pre_open_915_920': "order_time >= 91500000 AND order_time < 92000000",
        'pre_open_920_925': "order_time >= 92000000 AND order_time < 92500000",
        'early_930_1000': "order_time >= 93000000 AND order_time < 100000000",
        'main_1000_1430': "order_time >= 100000000 AND order_time < 143000000",
        'late_1430_1457': "order_time >= 143000000 AND order_time < 145700000",
        'close_1457_1500': "order_time >= 145700000 AND order_time < 150000000",
        'continue_930_1457':"order_time >= 93000000 AND order_time < 145700000"
    }

    # 创建的表名列表
    created_tables = []
    
    # 分位数类型列表
    percentile_types = ['p01_p99', 'p05_p95']

    try:
        # 为每个时间段和买卖方向创建临时表
        for interval_name, time_condition in time_intervals.items():
            # 买方订单
            bid_query = f"""
            SELECT 
                security_code,
                order_side, 
                order_type,
                order_details,
                order_price,
                order_price_adj,
                order_volume,
                order_time
            FROM '{order_pth}'
            WHERE order_type = 'A'
                AND order_details = 'L'
                AND order_side = 1
                AND {time_condition}
                AND order_price > 0
            """
            
            # 卖方订单
            ask_query = f"""
            SELECT 
                security_code,
                order_side, 
                order_type,
                order_details,
                order_price,
                order_price_adj,
                order_volume,
                order_time
            FROM '{order_pth}'
            WHERE order_type = 'A'
                AND order_details = 'L'
                AND order_side = -1
                AND {time_condition}
                AND order_price > 0
            """
            
            # 检查买方订单是否有数据
            check_bid_query = f"""
            SELECT COUNT(*) AS count FROM ({bid_query}) t
            """
            bid_count = conn.execute(check_bid_query).fetchone()[0]
            
            if bid_count > 0:
                # 创建买方基础表
                conn.execute(f"CREATE TEMPORARY TABLE bid_base_{interval_name} AS {bid_query}")
                
                # 计算买方价格分位数
                bid_percentile_query = f"""
                CREATE TEMPORARY TABLE bid_percentiles_{interval_name} AS
                SELECT 
                    security_code,
                    PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY order_price) AS p01,
                    PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY order_price) AS p99,

                FROM bid_base_{interval_name}
                GROUP BY security_code
                """
                conn.execute(bid_percentile_query)
                
                # 创建买方P01-P99缩尾表
                bid_winsorized_query_p01_p99 = f"""
                CREATE TEMPORARY TABLE bid_{interval_name}_p01_p99 AS
                SELECT 
                    b.security_code,
                    b.order_side,
                    b.order_type,
                    b.order_details,
                    CASE 
                        WHEN b.order_price < p.p01 THEN p.p01
                        WHEN b.order_price > p.p99 THEN p.p99
                        ELSE b.order_price
                    END AS order_price,
                    b.order_price_adj,
                    b.order_volume,
                    b.order_time
                FROM bid_base_{interval_name} b
                JOIN bid_percentiles_{interval_name} p ON b.security_code = p.security_code
                """
                conn.execute(bid_winsorized_query_p01_p99)
                created_tables.append(f'bid_{interval_name}_p01_p99')
                
            
            # 检查卖方订单是否有数据
            check_ask_query = f"""
            SELECT COUNT(*) AS count FROM ({ask_query}) t
            """
            ask_count = conn.execute(check_ask_query).fetchone()[0]
            
            if ask_count > 0:
                # 创建卖方基础表
                conn.execute(f"CREATE TEMPORARY TABLE ask_base_{interval_name} AS {ask_query}")
                
                # 计算卖方价格分位数
                ask_percentile_query = f"""
                CREATE TEMPORARY TABLE ask_percentiles_{interval_name} AS
                SELECT 
                    security_code,
                    PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY order_price) AS p01,
                    PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY order_price) AS p99,

                FROM ask_base_{interval_name}
                GROUP BY security_code
                """
                conn.execute(ask_percentile_query)
                
                # 创建卖方P01-P99缩尾表
                ask_winsorized_query_p01_p99 = f"""
                CREATE TEMPORARY TABLE ask_{interval_name}_p01_p99 AS
                SELECT 
                    a.security_code,
                    a.order_side,
                    a.order_type,
                    a.order_details,
                    CASE 
                        WHEN a.order_price < p.p01 THEN p.p01
                        WHEN a.order_price > p.p99 THEN p.p99
                        ELSE a.order_price
                    END AS order_price,
                    a.order_price_adj,
                    a.order_volume,
                    a.order_time
                FROM ask_base_{interval_name} a
                JOIN ask_percentiles_{interval_name} p ON a.security_code = p.security_code
                """
                conn.execute(ask_winsorized_query_p01_p99)
                created_tables.append(f'ask_{interval_name}_p01_p99')
                
                # # 创建卖方P05-P95缩尾表
                # ask_winsorized_query_p05_p95 = f"""
                # CREATE TEMPORARY TABLE ask_{interval_name}_p05_p95 AS
                # SELECT 
                #     a.security_code,
                #     a.order_side,
                #     a.order_type,
                #     a.order_details,
                #     CASE 
                #         WHEN a.order_price < p.p05 THEN p.p05
                #         WHEN a.order_price > p.p95 THEN p.p95
                #         ELSE a.order_price
                #     END AS order_price,
                #     a.order_price_adj,
                #     a.order_volume,
                #     a.order_time
                # FROM ask_base_{interval_name} a
                # JOIN ask_percentiles_{interval_name} p ON a.security_code = p.security_code
                # """
                # conn.execute(ask_winsorized_query_p05_p95)
                # created_tables.append(f'ask_{interval_name}_p05_p95')

        # 计算所有因子
        result_df = None
        
        for factor_name in created_tables:
            # 提取时间段和分位数类型
            parts = factor_name.split('_')
            side = parts[0]  # bid 或 ask
            interval_name = '_'.join(parts[1:-2])  # 时间段名称
            percentile_type = parts[-2] + '_' + parts[-1]  # p03_p97 
            
            # 检查表是否有足够的数据计算因子
            check_query = f"""
            SELECT COUNT(*) AS count FROM {factor_name} 
            WHERE order_price IS NOT NULL
            """
            count = conn.execute(check_query).fetchone()[0]
            
            if count <= 1:  # 需要至少两条记录才能计算分歧度
                continue
                
            try:
                # 计算订单加权平均对数价格
                factor_query = f"""
                WITH {factor_name}_stats AS (
                    SELECT 
                        security_code,
                        SUM(order_volume) AS total_volume,
                        SUM(LN(order_price) * order_volume) / SUM(order_volume) AS weighted_log_price
                    FROM {factor_name}
                    WHERE order_price IS NOT NULL AND order_price > 0  -- 确保只使用有效价格
                    GROUP BY security_code
                    HAVING COUNT(*) > 1  -- 确保每个证券至少有两条记录
                ),

                {factor_name}_price_divergence AS (
                    SELECT 
                        o.security_code,
                        SQRT(
                            SUM(POWER(LN(o.order_price) - s.weighted_log_price, 2) * o.order_volume) / 
                            ANY_VALUE(s.total_volume)
                        ) AS {side}_{interval_name}_{percentile_type}
                    FROM {factor_name} o
                    JOIN {factor_name}_stats s ON o.security_code = s.security_code
                    WHERE o.order_price IS NOT NULL AND o.order_price > 0  -- 确保只使用有效价格
                    GROUP BY o.security_code
                )

                SELECT 
                    security_code,
                    {side}_{interval_name}_{percentile_type}
                FROM {factor_name}_price_divergence
                """
                
                current_df = conn.execute(factor_query).fetchdf()
                
                if len(current_df) > 0:
                    if result_df is None:
                        result_df = current_df
                    else:
                        result_df = result_df.merge(
                            current_df,
                            on='security_code',
                            how='outer'
                        )
            except Exception as e:
                print(f"计算因子 {factor_name} 时出错: {str(e)}")
                continue
                
    except Exception as e:
        print(f"处理日期 {date_str} 时出错: {str(e)}")
        raise
    finally:
        conn.close()
    
    # 如果没有结果，返回空DataFrame
    if result_df is None or len(result_df) == 0:
        columns = ['security_code']
        return pd.DataFrame(columns=columns)
    
    return result_df
    



In [3]:
def process_single_date(trading_date, factor_name):
    
    """处理单个交易日的函数"""
   
    date_str = trading_date.strftime('%Y%m%d')
    output_dir = f"./factors/{factor_name}"
    file_path = f"{output_dir}/{date_str}.parquet"
    
    # 如果文件已存在，跳过处理
    if os.path.exists(file_path):
        print(f"文件已存在，跳过: {date_str}")
        return date_str, True, "已存在"
    
    # 检查原始数据文件是否存在
    data_file = f"/data/cephfs/order/{date_str}.parquet"
    if not os.path.exists(data_file):
        print(f"跳过日期 {date_str}: 原始数据文件不存在")
        return date_str, False, "数据文件不存在"
    
    try:
        daily_factor_df = calculate_daily_factor(trading_date)
        
        daily_factor_df['date'] = date_str

        daily_factor_df.to_parquet(file_path, index=False)
        print(f"已生成因子文件: {file_path}")
        return date_str, True, "成功"
        
    except Exception as e:
        error_msg = str(e)
        print(f"处理日期 {date_str} 时出错: {error_msg}")
        return date_str, False, error_msg


def derive_daily_factor(start_date, end_date, factor_name, num_processes=16):
    start_time = time.time()
    
    trading_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 创建输出目录
    output_dir = f"./factors/{factor_name}"
    os.makedirs(output_dir, exist_ok=True)
    
    # 创建进程池
    pool = mp.Pool(processes=num_processes)
    
    # 创建带有固定参数的处理函数
    process_date_with_args = partial(process_single_date, factor_name=factor_name)
    
    # 提交所有任务到进程池并获取结果
    print(f"开始使用 {num_processes} 个进程并行处理 {len(trading_dates)} 个交易日...")
    results = pool.map(process_date_with_args, trading_dates)
    
    # 关闭进程池
    pool.close()
    pool.join()
    
    # 分析处理结果
    processed_dates = [date_str for date_str, success, _ in results if success]
    skipped_dates = [date_str for date_str, success, _ in results if not success]
    
    # 打印处理结果摘要
    print(f"\n处理完成:")
    print(f"成功处理 {len(processed_dates)} 个日期")
    print(f"跳过 {len(skipped_dates)} 个日期")

    return output_dir


In [4]:
def main():
    start_date = '2016-06-20'
    end_date = '2025-06-30'
    factor_name = '1_99_percent_price_divergence'
    factor_dir = derive_daily_factor (start_date, end_date, factor_name)

main()

文件已存在，跳过: 20160811文件已存在，跳过: 20160620文件已存在，跳过: 20170307跳过日期 20161002: 原始数据文件不存在文件已存在，跳过: 20170810跳过日期 20170114: 原始数据文件不存在文件已存在，跳过: 20161123文件已存在，跳过: 20170428文件已存在，跳过: 20170619跳过日期 20171001: 原始数据文件不存在




文件已存在，跳过: 20160812文件已存在，跳过: 20171122

跳过日期 20180113: 原始数据文件不存在文件已存在，跳过: 20160621

文件已存在，跳过: 20170308
文件已存在，跳过: 20180306文件已存在，跳过: 20170811
跳过日期 20180618: 原始数据文件不存在跳过日期 20161003: 原始数据文件不存在文件已存在，跳过: 20180809文件已存在，跳过: 20180427

文件已存在，跳过: 20161124跳过日期 20170115: 原始数据文件不存在
跳过日期 20170429: 原始数据文件不存在跳过日期 20171002: 原始数据文件不存在

文件已存在，跳过: 20170620
文件已存在，跳过: 20171123


文件已存在，跳过: 20160622
跳过日期 20180114: 原始数据文件不存在


文件已存在，跳过: 20170309跳过日期 20160813: 原始数据文件不存在
文件已存在，跳过: 20180307跳过日期 20170812: 原始数据文件不存在跳过日期 20161004: 原始数据文件不存在
跳过日期 20180428: 原始数据文件不存在文件已存在，跳过: 20180619

文件已存在，跳过: 20170116
文件已存在，跳过: 20180810
跳过日期 20171003: 原始数据文件不存在文件已存在，跳过: 20170621



文件已存在，跳过: 20171124文件已存在，跳过: 20161125

跳过日期 20170430: 原始数据文件不存在文件已存在，跳过: 20160623
文件已存在，跳过: 20180115
跳过日期 20160814: 原始数据文件不存在

文件已存在，跳过: 20180308跳过日期 2017081





跳过日期 20191019: 原始数据文件不存在
文件已存在，跳过: 20191209

跳过日期 20200705: 原始数据文件不存在文件已存在，跳过: 20190327文件已存在，跳过: 20200515文件已存在，跳过: 20181022跳过日期 20190204: 原始数据文件不存在文件已存在，跳过: 20190708文件已存在，跳过: 20190516跳过日期 20200201: 原始数据文件不存在
跳过日期 20190831: 原始数据文件不存在
文件已存在，跳过: 20181212
跳过日期 20201018: 原始数据文件不存在







跳过日期 20191020: 原始数据文件不存在
文件已存在，跳过: 20191210
文件已存在，跳过: 20200706文件已存在，跳过: 20181023跳过日期 20200516: 原始数据文件不存在文件已存在，跳过: 20190328跳过日期 20190205: 原始数据文件不存在文件已存在，跳过: 20190709文件已存在，跳过: 20190517跳过日期 20200202: 原始数据文件不存在
跳过日期 20190901: 原始数据文件不存在文件已存在，跳过: 20181213文件已存在，跳过: 20201019








文件已存在，跳过: 20191021


文件已存在，跳过: 20191211文件已存在，跳过: 20200707文件已存在，跳过: 20181024文件已存在，跳过: 20190329跳过日期 20200517: 原始数据文件不存在文件已存在，跳过: 20190710跳过日期 20190206: 原始数据文件不存在跳过日期 20190518: 原始数据文件不存在文件已存在，跳过: 20200203
文件已存在，跳过: 20190902文件已存在，跳过: 20181214文件已存在，跳过: 20201020








文件已存在，跳过: 20191022


文件已存在，跳过: 20200708文件已存在，跳过: 20181025文件已存在，跳过: 20191212文件已存在，跳过: 20200518跳过日期 20190330: 原始数据文件不存在跳过日期 20190519: 原始数据文件不存在文件已存在，跳过: 20190711
文件已存在，跳过: 2

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

文件已存在，跳过: 20220328
文件已存在，跳过: 20220208文件已存在，跳过: 20220519


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

已生成因子文件: ./factors/1_99_percent_price_divergence/20241112.parquet

处理完成:
成功处理 2192 个日期
跳过 1106 个日期


In [1]:
import os
import glob
import pandas as pd
import numpy as np
import duckdb

def standardize_parquet_files(factor_dir):
    """
    读取目录中的所有Parquet文件，确保它们具有相同的列结构，然后重新保存。
    
    参数:
    factor_dir (str): 包含Parquet文件的目录路径
    
    返回:
    int: 处理的文件数量
    """
    # 获取目录中的所有parquet文件
    parquet_files = glob.glob(os.path.join(factor_dir, "*.parquet"))
    if not parquet_files:
        print(f"在目录 {factor_dir} 中未找到Parquet文件")
        return 0
    
    print(f"在目录 {factor_dir} 中找到 {len(parquet_files)} 个Parquet文件")

    all_columns = set(['date','security_code', 'bid_pre_open_915_920_p01_p99',
       'bid_pre_open_915_920_p05_p95', 'ask_pre_open_915_920_p01_p99',
       'ask_pre_open_915_920_p05_p95', 'bid_pre_open_920_925_p01_p99',
       'bid_pre_open_920_925_p05_p95', 'ask_pre_open_920_925_p01_p99',
       'ask_pre_open_920_925_p05_p95', 'bid_early_930_1000_p01_p99',
       'bid_early_930_1000_p05_p95', 'ask_early_930_1000_p01_p99',
       'ask_early_930_1000_p05_p95', 'bid_main_1000_1430_p01_p99',
       'bid_main_1000_1430_p05_p95', 'ask_main_1000_1430_p01_p99',
       'ask_main_1000_1430_p05_p95', 'bid_late_1430_1457_p01_p99',
       'bid_late_1430_1457_p05_p95', 'ask_late_1430_1457_p01_p99',
       'ask_late_1430_1457_p05_p95', 'bid_close_1457_1500_p01_p99',
       'bid_close_1457_1500_p05_p95', 'ask_close_1457_1500_p01_p99',
       'ask_close_1457_1500_p05_p95', 'bid_continue_930_1457_p01_p99',
       'bid_continue_930_1457_p05_p95', 'ask_continue_930_1457_p01_p99',
       'ask_continue_930_1457_p05_p95'])
    
    # # 收集所有文件中的所有列名
    # all_columns = set(['security_code', 'date', 
    #     'pre_open_915_920', 
    #     'pre_open_920_925', 
    #     'early_930_1000', 
    #     'main_1000_1430', 
    #     'late_1430_1457', 
    #     'close_1457_1500',
    #     'continue_930_1457'])  # 这些列必须存在
    
    # # 定义所有可能的因子列名
    # time_intervals = [
    #     'pre_open_915_920', 
    #     'pre_open_920_925', 
    #     'early_930_1000', 
    #     'main_1000_1430', 
    #     'late_1430_1457', 
    #     'close_1457_1500',
    #     'continue_930_1457'
    # ]
    
    # for interval in time_intervals:
    #     all_columns.add(f"bid_{interval}")
    
    # 处理每个文件
    for i, file_path in enumerate(parquet_files):
        try:
            # 读取Parquet文件
            df = pd.read_parquet(file_path)
            
            # 添加缺失的列
            missing_columns = all_columns - set(df.columns)
            for col in missing_columns:
                df[col] = np.nan
            
            # 如果缺少factor_name列，添加它
            if 'factor_name' not in df.columns:
                df['factor_name'] = "3_97_percent_price_divergence"
            
            # 重新保存文件
            df.to_parquet(file_path, index=False)
            
            if (i+1) % 10 == 0 or i+1 == len(parquet_files):
                print(f"已处理 {i+1}/{len(parquet_files)} 个文件")
                
        except Exception as e:
            print(f"处理文件 {file_path} 时出错: {str(e)}")
    
    print("所有文件处理完成")
    return len(parquet_files)


In [12]:
def filter_and_pivot_factor(all_factor_data, factor_name):
    """
    筛选特定因子列且该列不为NaN的数据，并进行透视
    
    参数:
    all_factor_data: 原始数据DataFrame
    factor_name: 要筛选的因子列名称，例如'bid_pre_open_915_920_p01_p99'
    
    返回:
    透视后的DataFrame，索引为date，列为security_code
    """
    # 步骤1: 筛选必要的列
    filtered_data = all_factor_data[['date', 'security_code', factor_name]]
    
    # 步骤2: 去除因子列为NaN的行
    filtered_data = filtered_data.dropna(subset=[factor_name])
    
    # 步骤3: 检查是否存在重复的(date, security_code)组合
    duplicates = filtered_data.duplicated(subset=['date', 'security_code'], keep=False)
    if duplicates.any():
        print(f"警告: 发现{duplicates.sum()}条重复记录，将保留第一条")
        # 如果有重复，保留第一条
        filtered_data = filtered_data.drop_duplicates(subset=['date', 'security_code'], keep='first')

    output_path = f"/data/home/lexuanchen/Factors/Order/Signal/1_100_Percent_Raw_Price_Divergence_2"
    os.makedirs(output_path, exist_ok=True)
    output_file_path = f"{output_path}/Raw_{factor_name}.csv"
    filtered_data.to_csv(output_file_path)
    print(f"已成功保存因子原文件：{factor_name}")
    
    # 步骤4: 透视表，将security_code作为列，date作为索引
    pivot_data = filtered_data.pivot(index='date', columns='security_code', values=factor_name)
    
    return pivot_data


In [10]:
def calculate_rolling_20d_avg(factor_names, factor_dir, start_date=None, end_date=None, min_window=5):
    
    print(factor_names)
    
    if not isinstance(factor_names, list):
        factor_names = [factor_names]
    
    parquet_pattern = os.path.join(factor_dir, "*.parquet")
    parquet_files = glob.glob(parquet_pattern)
    print(f"在目录 {factor_dir} 中找到 {len(parquet_files)} 个Parquet文件")
    

    conn = duckdb.connect(database=':memory:')
    all_factor_data = conn.execute(f"""
        SELECT * FROM read_parquet('{parquet_pattern}')
    """).fetchdf()
    conn.close()
    
    

    missing_factors = [f for f in factor_names if f not in all_factor_data.columns]
    if missing_factors:
        raise ValueError(f"在Parquet文件中未找到以下因子列: {', '.join(missing_factors)}")
    
    all_factor_data['date'] = pd.to_datetime(all_factor_data['date'])

    all_factor_data = all_factor_data.sort_values(['date', 'security_code'])
    
    # 筛选日期范围
    if start_date:
        start_date = pd.to_datetime(start_date)
        all_factor_data = all_factor_data[all_factor_data['date'] >= start_date]
    if end_date:
        end_date = pd.to_datetime(end_date)
        all_factor_data = all_factor_data[all_factor_data['date'] <= end_date]

    # 获取所有个股代码
    all_securities = all_factor_data['security_code'].unique()

    result_df = all_factor_data[['date', 'security_code']].copy()

    # 为每个因子计算滚动平均
    for factor_name in factor_names:
        print(f"\n处理因子: {factor_name}")
        
        #转置成宽表
        pivot_data = filter_and_pivot_factor(all_factor_data,factor_name)

        # 对宽表直接应用rolling
        rolling_avg = pivot_data.rolling(window=20, min_periods=min_window).mean()

        # 将结果转换回长格式
        factor_df = rolling_avg.stack().reset_index()
        factor_df.columns = ['date', 'security_code', factor_name]
        factor_df[factor_name] = - factor_df[factor_name]

        # 计算每日因子覆盖率
        # 计算每个日期非NaN的因子值数量
        non_nan_counts = factor_df.dropna(subset=[factor_name]).groupby('date').size()

        # 计算覆盖率
        coverage = non_nan_counts / len(all_securities)
        print(f"\n{factor_name}因子覆盖率统计: 平均={coverage.mean():.2f}, 最小={coverage.min():.2f}")

        result_df = pd.merge(
            result_df, 
            factor_df,
            on=['date', 'security_code'],
            how='left'
        )
    
    return result_df

In [5]:
def main():
    start_date = '2016-06-20'
    end_date = '2025-06-30'
    
    # 定义时间段列表
    time_intervals = [
        'pre_open_915_920',
        'pre_open_920_925',
        'early_930_1000',
        'main_1000_1430',
        'late_1430_1457',
        'close_1457_1500',
        'continue_930_1457'
    ]
    
    # # 定义因子类型
    # factor_types = [
    #     'bid_p01_p99',
    #     'bid_p05_p95',
    #     'ask_p01_p99',
    #     'ask_p05_p95'
    # ]

        # 定义因子类型
    factor_types = [
        'bid_p03_p97',
        'ask_p03_p97',
    ]
    
    # 生成所有因子名称列表
    all_factors = []
    for factor_type in factor_types:
        side = factor_type.split('_')[0]  # bid 或 ask
        percentile = '_'.join(factor_type.split('_')[1:])  # p01_p99 或 p05_p95
        for interval in time_intervals:
            all_factors.append(f"{side}_{interval}_{percentile}")
    
    # 为每种因子类型创建单独的目录
    factor_dirs = {}
    for factor_type in factor_types:
        factor_name = f"3_97_percent_{factor_type}_price_divergence"
        factor_dir = f"./factors/3_97_percent_price_divergence"
        factor_dirs[factor_type] = factor_dir
        # print(f"{factor_type} 因子文件已生成在目录: {factor_dir}")
        
        # 统一化列名
    # standardize_parquet_files(factor_dir)
    
    # 为每种因子类型计算滚动平均值
    for factor_type, factor_dir in factor_dirs.items():
        # 获取该类型的因子列表
        type_factors = [f for f in all_factors if f.startswith(factor_type.split('_')[0]) and f.endswith('_'.join(factor_type.split('_')[1:]))]
        
        # 计算滚动平均
        result_df = calculate_rolling_20d_avg(type_factors, factor_dir, start_date, end_date, min_window=5)
        
        # 保存结果为CSV
        output_path = f"/data/home/lexuanchen/Factors/Order/Signal/3_97_Percent_{factor_type.upper()}_Price_Divergence_2"
        os.makedirs(output_path, exist_ok=True)
        
        print(f"{factor_type} 因子共计 {len(result_df)} 条记录")
        print("\n数据预览:")
        print(result_df.head())
        
        # 保存每个因子的CSV文件
        for factor in type_factors:
            output_file_path = f"{output_path}/20d_{factor}.csv"
            factor_df = result_df[['date', 'security_code', factor]].dropna(subset=[factor])
            factor_df.to_csv(output_file_path, index=False)

# 执行主函数
if __name__ == "__main__":
    main()


['bid_pre_open_915_920_p03_p97', 'bid_pre_open_920_925_p03_p97', 'bid_early_930_1000_p03_p97', 'bid_main_1000_1430_p03_p97', 'bid_late_1430_1457_p03_p97', 'bid_close_1457_1500_p03_p97', 'bid_continue_930_1457_p03_p97']
在目录 ./factors/3_97_percent_price_divergence 中找到 2164 个Parquet文件


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


处理因子: bid_pre_open_915_920_p03_p97
已成功保存因子原文件：bid_pre_open_915_920_p03_p97

bid_pre_open_915_920_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_pre_open_920_925_p03_p97
已成功保存因子原文件：bid_pre_open_920_925_p03_p97

bid_pre_open_920_925_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_early_930_1000_p03_p97
已成功保存因子原文件：bid_early_930_1000_p03_p97

bid_early_930_1000_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_main_1000_1430_p03_p97
已成功保存因子原文件：bid_main_1000_1430_p03_p97

bid_main_1000_1430_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_late_1430_1457_p03_p97
已成功保存因子原文件：bid_late_1430_1457_p03_p97

bid_late_1430_1457_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_close_1457_1500_p03_p97
已成功保存因子原文件：bid_close_1457_1500_p03_p97

bid_close_1457_1500_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_continue_930_1457_p03_p97
已成功保存因子原文件：bid_continue_930_1457_p03_p97

bid_continue_930_1457_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47
bid_p03_p97 因子共计 8852399 条记录

数据预览:
        date security_code  bid_pre_open_915_920_p03_p97  \
0 201

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


处理因子: ask_pre_open_915_920_p03_p97
已成功保存因子原文件：ask_pre_open_915_920_p03_p97

ask_pre_open_915_920_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_pre_open_920_925_p03_p97
已成功保存因子原文件：ask_pre_open_920_925_p03_p97

ask_pre_open_920_925_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_early_930_1000_p03_p97
已成功保存因子原文件：ask_early_930_1000_p03_p97

ask_early_930_1000_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_main_1000_1430_p03_p97
已成功保存因子原文件：ask_main_1000_1430_p03_p97

ask_main_1000_1430_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_late_1430_1457_p03_p97
已成功保存因子原文件：ask_late_1430_1457_p03_p97

ask_late_1430_1457_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_close_1457_1500_p03_p97
已成功保存因子原文件：ask_close_1457_1500_p03_p97

ask_close_1457_1500_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_continue_930_1457_p03_p97
已成功保存因子原文件：ask_continue_930_1457_p03_p97

ask_continue_930_1457_p03_p97因子覆盖率统计: 平均=0.77, 最小=0.47
ask_p03_p97 因子共计 8852399 条记录

数据预览:
        date security_code  ask_pre_open_915_920_p03_p97  \
0 201

In [ ]:
import pandas as pd
path = "/data/home/lexuanchen/Factors/Order/Signal/1_100_Percent_BID_P01_P99_Price_Divergence/bid_main_1000_1430_p01_p99.csv"
df = pd.read_csv(path)

df.head()

In [6]:
def filter_data(all_factor_data, factor_name, factor_type):
    """
    筛选特定因子列且该列不为NaN的数据，并进行透视
    
    参数:
    all_factor_data: 原始数据DataFrame
    factor_name: 要筛选的因子列名称，例如'bid_pre_open_915_920_p01_p99'
    
    返回:
    透视后的DataFrame，索引为date，列为security_code
    """
    # 步骤1: 筛选必要的列
    mask = all_factor_data['factor_type'] == factor_type
    filtered_data = all_factor_data[mask][['date', 'security_code', factor_name]]
    
    # 步骤2: 去除因子列为NaN的行
    filtered_data = filtered_data.dropna(subset=[factor_name])
    
    # 步骤3: 检查是否存在重复的(date, security_code)组合
    duplicates = filtered_data.duplicated(subset=['date', 'security_code'], keep=False)
    if duplicates.any():
        print(f"警告: 发现{duplicates.sum()}条重复记录，将保留第一条")
        # 如果有重复，保留第一条
        filtered_data = filtered_data.drop_duplicates(subset=['date', 'security_code'], keep='first')
    
    
    return filtered_data


In [13]:
def main():
    start_date = '2016-06-20'
    end_date = '2025-06-30'
    
    # 定义时间段列表
    time_intervals = [
        'pre_open_915_920',
        'pre_open_920_925',
        'early_930_1000',
        'main_1000_1430',
        'late_1430_1457',
        'close_1457_1500',
        'continue_930_1457'
    ]
    
    # 定义因子类型
    factor_types = [
        'bid_p01_p99',
        # 'bid_p05_p95',
        'ask_p01_p99'
        #, 'ask_p05_p95'
    ]
    
    # 生成所有因子名称列表
    all_factors = []
    for factor_type in factor_types:
        side = factor_type.split('_')[0]  # bid 或 ask
        percentile = '_'.join(factor_type.split('_')[1:])  # p01_p99 或 p05_p95
        for interval in time_intervals:
            all_factors.append(f"{side}_{interval}_{percentile}")
    
    # 为每种因子类型创建单独的目录
    factor_dirs = {}
    for factor_type in factor_types:
        # factor_name = f"1_100_percent_{factor_type}_price_divergence"
        factor_dir = f"./factors/1_99_percent_price_divergence"
        factor_dirs[factor_type] = factor_dir
        # print(f"{factor_type} 因子文件已生成在目录: {factor_dir}")
        
        # 统一化列名
        # standardize_parquet_files(factor_dir)
    
    # 为每种因子类型计算滚动平均值
    for factor_type, factor_dir in factor_dirs.items():
        # 获取该类型的因子列表
        type_factors = [f for f in all_factors if f.startswith(factor_type.split('_')[0]) and f.endswith('_'.join(factor_type.split('_')[1:]))]

        parquet_pattern = os.path.join(factor_dir, "*.parquet")
        parquet_files = glob.glob(parquet_pattern)
        print(f"在目录 {factor_dir} 中找到 {len(parquet_files)} 个Parquet文件")
        

        conn = duckdb.connect(database=':memory:')
        all_factor_data = conn.execute(f"""
            SELECT * FROM read_parquet('{parquet_pattern}')
        """).fetchdf()
        conn.close()
        

        all_factor_data['date'] = pd.to_datetime(all_factor_data['date'])

        all_factor_data = all_factor_data.sort_values(['date', 'security_code'])

        
        # 保存结果为CSV
        output_path = f"/data/home/lexuanchen/Factors/Order/Signal/1_99_Percent_{factor_type.upper()}_Price_Divergence_2"
        os.makedirs(output_path, exist_ok=True)
        
        print(f"{factor_type} 因子共计 {len(all_factor_data)} 条记录")
        print("\n数据预览:")
        print(all_factor_data.head())
        
        # 保存每个因子的CSV文件
        for factor in type_factors:
            result_df = filter_data(all_factor_data, factor, factor_type)
            output_file_path = f"{output_path}/raw_{factor}.csv"
            factor_df = result_df[['date', 'security_code', factor]].dropna(subset=[factor])
            factor_df.to_csv(output_file_path, index=False)

# 执行主函数
if __name__ == "__main__":
    main()


在目录 ./factors/1_99_percent_price_divergence 中找到 2192 个Parquet文件


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

bid_p01_p99 因子共计 8992552 条记录

数据预览:
  security_code  bid_pre_open_915_920_p01_p99  ask_pre_open_915_920_p01_p99  \
0        000001                      0.025253                      0.033544   
1        000005                      0.034132                      0.029355   
2        000006                      0.034470                      0.030007   
3        000008                      0.024734                      0.031154   
4        000009                      0.032511                      0.029375   

   bid_pre_open_920_925_p01_p99  ask_pre_open_920_925_p01_p99  \
0                      0.024399                      0.026576   
1                      0.025580                      0.039455   
2                      0.007618                      0.023609   
3                      0.015137                      0.038711   
4                      0.027177                      0.028236   

   bid_early_930_1000_p01_p99  ask_early_930_1000_p01_p99  \
0                    0.012684        

KeyError: 'factor_type'

In [14]:
def main():
    start_date = '2016-06-20'
    end_date = '2025-06-30'
    
    # 定义时间段列表
    time_intervals = [
        'pre_open_915_920',
        'pre_open_920_925',
        'early_930_1000',
        'main_1000_1430',
        'late_1430_1457',
        'close_1457_1500',
        'continue_930_1457'
    ]
    
    # # 定义因子类型
    # factor_types = [
    #     'bid_p01_p99',
    #     'bid_p05_p95',
    #     'ask_p01_p99',
    #     'ask_p05_p95'
    # ]

        # 定义因子类型
    factor_types = [
        'bid_p01_p99',
        'ask_p01_p99',
    ]
    
    # 生成所有因子名称列表
    all_factors = []
    for factor_type in factor_types:
        side = factor_type.split('_')[0]  # bid 或 ask
        percentile = '_'.join(factor_type.split('_')[1:])  # p01_p99 或 p05_p95
        for interval in time_intervals:
            all_factors.append(f"{side}_{interval}_{percentile}")
    
    # 为每种因子类型创建单独的目录
    factor_dirs = {}
    for factor_type in factor_types:
        factor_name = f"1_99_percent_{factor_type}_price_divergence"
        factor_dir = f"./factors/1_99_percent_price_divergence"
        factor_dirs[factor_type] = factor_dir
        # print(f"{factor_type} 因子文件已生成在目录: {factor_dir}")
        
        # 统一化列名
    # standardize_parquet_files(factor_dir)
    
    # 为每种因子类型计算滚动平均值
    for factor_type, factor_dir in factor_dirs.items():
        # 获取该类型的因子列表
        type_factors = [f for f in all_factors if f.startswith(factor_type.split('_')[0]) and f.endswith('_'.join(factor_type.split('_')[1:]))]
        
        # 计算滚动平均
        result_df = calculate_rolling_20d_avg(type_factors, factor_dir, start_date, end_date, min_window=5)
        
        # 保存结果为CSV
        output_path = f"/data/home/lexuanchen/Factors/Order/Signal/1_99_Percent_{factor_type.upper()}_Price_Divergence_2"
        os.makedirs(output_path, exist_ok=True)
        
        print(f"{factor_type} 因子共计 {len(result_df)} 条记录")
        print("\n数据预览:")
        print(result_df.head())
        
        # 保存每个因子的CSV文件
        for factor in type_factors:
            output_file_path = f"{output_path}/20d_{factor}.csv"
            factor_df = result_df[['date', 'security_code', factor]].dropna(subset=[factor])
            factor_df.to_csv(output_file_path, index=False)

# 执行主函数
if __name__ == "__main__":
    main()


['bid_pre_open_915_920_p01_p99', 'bid_pre_open_920_925_p01_p99', 'bid_early_930_1000_p01_p99', 'bid_main_1000_1430_p01_p99', 'bid_late_1430_1457_p01_p99', 'bid_close_1457_1500_p01_p99', 'bid_continue_930_1457_p01_p99']
在目录 ./factors/1_99_percent_price_divergence 中找到 2192 个Parquet文件


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


处理因子: bid_pre_open_915_920_p01_p99
已成功保存因子原文件：bid_pre_open_915_920_p01_p99

bid_pre_open_915_920_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_pre_open_920_925_p01_p99
已成功保存因子原文件：bid_pre_open_920_925_p01_p99

bid_pre_open_920_925_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.46

处理因子: bid_early_930_1000_p01_p99
已成功保存因子原文件：bid_early_930_1000_p01_p99

bid_early_930_1000_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_main_1000_1430_p01_p99
已成功保存因子原文件：bid_main_1000_1430_p01_p99

bid_main_1000_1430_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_late_1430_1457_p01_p99
已成功保存因子原文件：bid_late_1430_1457_p01_p99

bid_late_1430_1457_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_close_1457_1500_p01_p99
已成功保存因子原文件：bid_close_1457_1500_p01_p99

bid_close_1457_1500_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: bid_continue_930_1457_p01_p99
已成功保存因子原文件：bid_continue_930_1457_p01_p99

bid_continue_930_1457_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47
bid_p01_p99 因子共计 8992552 条记录

数据预览:
        date security_code  bid_pre_open_915_920_p01_p99  \
0 201

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


处理因子: ask_pre_open_915_920_p01_p99
已成功保存因子原文件：ask_pre_open_915_920_p01_p99

ask_pre_open_915_920_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_pre_open_920_925_p01_p99
已成功保存因子原文件：ask_pre_open_920_925_p01_p99

ask_pre_open_920_925_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_early_930_1000_p01_p99
已成功保存因子原文件：ask_early_930_1000_p01_p99

ask_early_930_1000_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_main_1000_1430_p01_p99
已成功保存因子原文件：ask_main_1000_1430_p01_p99

ask_main_1000_1430_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_late_1430_1457_p01_p99
已成功保存因子原文件：ask_late_1430_1457_p01_p99

ask_late_1430_1457_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_close_1457_1500_p01_p99
已成功保存因子原文件：ask_close_1457_1500_p01_p99

ask_close_1457_1500_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47

处理因子: ask_continue_930_1457_p01_p99
已成功保存因子原文件：ask_continue_930_1457_p01_p99

ask_continue_930_1457_p01_p99因子覆盖率统计: 平均=0.77, 最小=0.47
ask_p01_p99 因子共计 8992552 条记录

数据预览:
        date security_code  ask_pre_open_915_920_p01_p99  \
0 201